In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import time
from pyproj import Proj, Transformer
from typing import Any, Dict, List
%matplotlib widget
import json
import socket
import json
import threading
from datetime import datetime
import tqdm

In [2]:
class Agent:
    def __init__(self, agent_id, parameters):
        # Initialize projection transformations for coordinate conversion
        self.utm_proj = Proj(proj='utm', zone=33, ellps='WGS84', preserve_units=False)
        self.wgs84_proj = Proj(proj='latlong', datum='WGS84')
        self.transformer = Transformer.from_proj(self.utm_proj, self.wgs84_proj)

        # Initialize agent properties
        self.id = agent_id  # Unique ID for each agent
        self.parameters = parameters
        self.state = self.initialize_state()
        self.position = np.array([500000.0, 5200000.0])  # Starting position (UTM coordinates)
        self.direction = 0  # Initial direction in degrees
        self.speed = 0  # Initial speed
        self.turning_angle = 0  # Initial turning angle
        self.neighbours = []  # Initialize an empty list for storing neighbors' IDs

    def initialize_state(self):
        # Initialize the state based on initial state probabilities
        states = self.parameters['states']
        state_probs = [self.parameters['initial_state_probabilities'][state] for state in states]
        initial_state = np.random.choice(states, p=state_probs)
        return initial_state
    
    def set_state(self,state):
        self.state = state

    def xy_to_wgs84(self, x, y):
        # Convert x/y (UTM coordinates) to lon/lat (WGS84 coordinates)
        lon, lat = self.transformer.transform(x, y)
        return lon, lat

    def get_ID(self):
        # Return the unique ID of the agent
        return str(self.id)

    def get_time(self):
        # Return the current time
        return str(datetime.now())

    def get_position(self):
        # Return the current position of the agent in WGS84 coordinates
        return self.xy_to_wgs84(self.position[0], self.position[1])

    def get_observed_position(self):
        # Return the observed position with added noise (simulating GPS error)
        x_error, y_error = np.random.multivariate_normal(self.parameters['Uxy'], self.parameters['Cxy'], 1)[0]
        return self.xy_to_wgs84(self.position[0] + x_error, self.position[1] + y_error)

    def get_direction(self):
        # Return the current direction of the agent
        return self.direction

    def get_state(self):
        # Return the current state of the agent
        return self.state

    def get_speed(self):
        # Return the current speed of the agent
        return self.speed

    def get_neighbours_speed(self, states_df):
        # Calculate the average speed of the agent's neighbors
        speed = sum(states_df[states_df['ID'] == neighbour]['speed'].iloc[0] for neighbour in self.neighbours)
        return speed / len(self.neighbours) if self.neighbours else 0

    def get_neighbours_direction(self, states_df):
        # Calculate the average direction of the agent's neighbors
        direction = sum(states_df[states_df['ID'] == neighbour]['direction'].iloc[0] for neighbour in self.neighbours)
        return direction / len(self.neighbours) if self.neighbours else 0

    def update_transition_probs(self):
        # Update the transition probabilities based on the current state
        self.transition_probs = self.parameters['state_transition_matrix'][self.state]

    def update_state(self, states_df):
        # Check if any of the neighbors are in the "Fleeing" state
        fleeing_neighbors = states_df[states_df['ID'].isin(self.neighbours) & (states_df['state'] == 'Fleeing')]
        
        if not fleeing_neighbors.empty:
            # If at least one neighbor is fleeing, the agent also changes to "Fleeing" state
            self.state = 'Fleeing'
        else:
            # Otherwise, update the state based on transition probabilities
            self.state = np.random.choice(self.parameters['states'], p=self.transition_probs)


    def update_state_params(self):
        # Update the parameters associated with the current state
        self.state_params = self.parameters['state_parameters'][self.state]

    def update_speed(self, states_df):
        # Update the speed of the agent considering neighbors' speed and randomness
        neighbours_speed = self.get_neighbours_speed(states_df)
        min_speed, max_speed = self.state_params['speed']
        self.speed = (neighbours_speed * self.parameters['weight_factor'] +
                      (1 - self.parameters['weight_factor']) * np.random.uniform(min_speed, max_speed))

    def update_direction(self, states_df):
        # Update the direction of the agent considering neighbors' direction and randomness
        neighbours_direction = self.get_neighbours_direction(states_df)
        if self.state in ['Grazing', 'Resting']:
            # Use a uniform distribution for turning angles
            min_angle, max_angle = self.state_params['turning_angle']
            self.turning_angle = (neighbours_direction * self.parameters['weight_factor'] +
                                  (1 - self.parameters['weight_factor']) * np.random.uniform(min_angle, max_angle))
        elif self.state in ['foraging', 'Fleeing']:
            # Use a Gaussian distribution for turning angles
            mean_angle, std_dev = self.state_params['turning_angle']
            self.turning_angle = (neighbours_direction * self.parameters['weight_factor'] +
                                  (1 - self.parameters['weight_factor']) * np.random.normal(mean_angle, std_dev))
        # Ensure the direction is within 0-360 degrees
        self.direction = (self.direction + self.turning_angle) % 360

    def update_parameters(self, states_df):
        # Update all parameters after each step
        self.update_transition_probs()
        self.update_state(states_df)
        self.update_state_params()
        self.update_speed(states_df)
        self.update_direction(states_df)

    def step_forward(self, states_df):
        # Move the agent forward by updating position based on speed and direction
        angle_rad = np.deg2rad(self.direction)
        delta_x = self.speed * np.cos(angle_rad) / self.parameters['fs']
        delta_y = self.speed * np.sin(angle_rad) / self.parameters['fs']
        self.position += np.array([delta_x, delta_y])
        # Update neighbors after moving
        self.neighbours = self.get_n_closest_neighbours(states_df, self.parameters['neighbours'])
        # Update all other parameters based on new position and neighbors
        self.update_parameters(states_df)

    def get_update(self):
        # Prepare and return a dictionary of the agent's current state and position
        out_dict = {
            'ID': self.get_ID(),
            'real_latitude': self.get_position()[0],
            'real_longitude': self.get_position()[1],
            'observed_latitude': self.get_observed_position()[0],
            'observed_longitude': self.get_observed_position()[1],
            'speed': self.speed,
            'direction': self.direction,
            'state': self.state,
            'time': self.get_time()
        }
        return out_dict

    def get_n_closest_neighbours(self, updates, N):
        # Calculate distances to all other agents and return the IDs of the N closest ones
        agent_position = updates[updates['ID'] == self.get_ID()][['real_latitude', 'real_longitude']].values[0]
        lat_diff = updates['real_latitude'].values - agent_position[0]
        lon_diff = updates['real_longitude'].values - agent_position[1]
        distances = lat_diff**2 + lon_diff**2  # Squared distances for comparison
        updates['distance'] = distances
        closest_agents = updates[updates['ID'] != self.get_ID()].nsmallest(N, 'distance')
        del updates['distance']  # Clean up the DataFrame
        return closest_agents['ID'].tolist()

In [3]:
PARAMETERS = {
    # Sampling rate and error covariance matrix definition
    'fs': 1,  # 1 samples per second
    
    'unique_herd_id': 'alp',
    'number_of_agents': 50,
    'neighbours': 5,

    # Weight factor definition
    # Weight for speed smoothing to control the influence of previous speed on the current speed
    'weight_factor': 0.8,

    # Mean vector for GPS error
    'Uxy': [0, 0],  # Mean vector for x and y GPS errors (assuming no bias)

    # Covariance matrix for GPS error
    # x and y are uncorrelated having a variance of 2 meters (standard deviation 2 meters)
    # Covariance matrix (4 meters^2 variance in x and y), indicating independent errors
    'Cxy': [[4, 0], [0, 4]],

    # Define the different states_df
    # Behavioral states_df: First 3 are normal, last one is anomalous
    'states': ['Grazing', 'Resting', 'foraging', 'Fleeing'],

    # Initial state probabilities
    'initial_state_probabilities': {
        'Grazing': 0.5,    # 50% chance to start in Grazing state
        'Resting': 0.5,    # 50% chance to start in Resting state
        'foraging': 0.0,  # 0% chance to start in foraging state
        'Fleeing': 0.0     # 0% chance to start in Fleeing state
    },

    # State transition matrix
    # Rows: current state, Columns: next state
    'state_transition_matrix': {
        # Probabilities of transitioning from Grazing to [Grazing, Resting, foraging, Fleeing]
        'Grazing': [0.89999, 0.05, 0.05, 0.00001],
        # Probabilities of transitioning from Resting to [Grazing, Resting, foraging, Fleeing]
        'Resting': [0.05, 0.89999, 0.05, 0.00001],
        # Probabilities of transitioning from foraging to [Grazing, Resting, foraging, Fleeing]
        'foraging': [0.05, 0.05, 0.89999, 0.00001],
        # Probabilities of transitioning from Fleeing to [Grazing, Resting, foraging, Fleeing]
        'Fleeing': [0.0003, 0.0003, 0.0003, 0.9991]
    },

    # Parameters for each state
    'state_parameters': {
        'Grazing': {
            'speed': (0.1, 0.5),  # Speed range in meters per second
            # Turning angle range (uniform distribution) since it is not directed
            'turning_angle': (-180, 180),
        },
        'Resting': {
            'speed': (0, 0.5),  # Speed range in meters per second
            # Turning angle range (uniform distribution) since it is not directed
            'turning_angle': (-180, 180),
        },
        'foraging': {
            'speed': (0.5, 1.5),  # Speed range in meters per second
            # Turning angle mean and standard deviation in degrees (Gaussian distribution)
            'turning_angle': (0, 20),
        },
        'Fleeing': {
            'speed': (2, 5),  # Speed range in meters per second
            # Turning angle mean and standard deviation in degrees (Gaussian distribution)
            'turning_angle': (0, 5),
        }
    }
}

In [4]:
update_list = []
update_df_list = []
update_df_past = 0
agents = [Agent(PARAMETERS['unique_herd_id'] + '_' + str(i), PARAMETERS) for i in range(PARAMETERS['number_of_agents'])]
[agent.set_state('Fleeing') for agent in agents]

for i in tqdm.tqdm(range(1000)):
    update_df = []
    for agent in agents:
        if i != 0:
            agent.step_forward(update_df_past)
        agent_update = agent.get_update()
        update_df.append(agent_update)
        update_list.append(agent_update)
    update_df_past = pd.DataFrame(update_df)
    update_df_list.append(update_df_past)
update_df_list = pd.concat(update_df_list).reset_index(drop=True)
update_df_list.to_parquet(f"simulations/anomaly.parquet")

100%|██████████| 1000/1000 [01:10<00:00, 14.20it/s]


# Simulate for training

In [ ]:
for simulation_num in range(0,25):
    update_list = []
    update_df_list = []
    update_df_past = 0
    agents = [Agent(PARAMETERS['unique_herd_id'] + '_' + str(i), PARAMETERS) for i in range(PARAMETERS['number_of_agents'])]
    for i in tqdm.tqdm(range(1000)):
        update_df = []
        for agent in agents:
            if i != 0:
                agent.step_forward(update_df_past)
            agent_update = agent.get_update()
            update_df.append(agent_update)
            update_list.append(agent_update)
        update_df_past = pd.DataFrame(update_df)
        update_df_list.append(update_df_past)
        
        if 'Fleeing' in update_df_past['state'].unique():
            update_df_list = pd.concat(update_df_list).reset_index(drop=True)
            update_df_list.to_parquet(f"simulations/simulation_{simulation_num}.parquet")
            break
    
    update_df_list = pd.concat(update_df_list).reset_index(drop=True)
    update_df_list.to_parquet(f"simulations/simulation_{simulation_num}.parquet")

In [ ]:
update_df_list['state'].value_counts()

In [ ]:
running_stats: Dict[str, Dict[str, List[float]]] = {}
WINDOW_SIZE = 100

for payload in update_list:
    global running_stats
    id_ = payload['ID']
    observed_latitude = payload['observed_latitude']
    observed_longitude = payload['observed_longitude']
    timestamp = payload['time']

    # Initialize the ID entry if it doesn't exist
    if id_ not in running_stats:
        running_stats[id_] = {
            "latitude": [],
            "longitude": [],
            "time": []
        }

    # Add the new data to the lists
    running_stats[id_]['latitude'].append(observed_latitude)
    running_stats[id_]['longitude'].append(observed_longitude)
    running_stats[id_]['time'].append(timestamp)

    # Ensure the lists maintain the fixed window size
    if len(running_stats[id_]['latitude']) > WINDOW_SIZE:
        running_stats[id_]['latitude'].pop(0)
        running_stats[id_]['longitude'].pop(0)
        running_stats[id_]['time'].pop(0)

    print(running_stats)

In [ ]:
plt.close('all')
plt.figure()
plt.plot(running_stats['alp_13']['longitude'])

In [ ]:
update['speed'] * 2

In [ ]:
plt.close('all')
plt.figure(figsize=[12, 6])
for id in update_df_list['ID'].unique():
    df = update_df_list[update_df_list['ID'] == id]

    # Define state colors
    state_colors = {
        'Grazing': 'green',
        'Resting': 'blue',
        'foraging': 'brown',
        'Fleeing': 'red'
    }

    # Plotting real and observed positions with different markers
    for state, color in state_colors.items():
        state_df = df[df['state'] == state]
        # Plot real positions with 'o' marker
        plt.scatter(state_df['real_latitude'], state_df['real_longitude'],
                    c=color, label=f'{state} (Real)', s=10, marker='o')
        # Plot observed positions with 'x' marker
        # plt.scatter(state_df['observed_x'], state_df['observed_y'], c=color, label=f'{state} (Observed)', s=10, marker='x')

plt.xlabel('latitude')
plt.ylabel('longitude')
plt.title('Scatter Plot of X and Y with State Colors')
# plt.legend()
plt.grid(True)
plt.show()

In [ ]:
update_df_list['state'].value_counts()

In [ ]:
plt.figure()
id = '3'
x = agents_data_dict[id]['observed_latitude']
y = agents_data_dict[id]['observed_longitude']
x -= x.mean()
x /= x.std()
y -= y.mean()
y /= y.std()

plt.plot(x)
plt.plot(y)